<a href="https://colab.research.google.com/github/Atsu-fuji/How-to-scrape-2ch-and-5ch-in-pyhon/blob/main/how_to_scrape_2ch_and_5ch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 匿名電子掲示板（2ちゃんねる・5ちゃんねる）の投稿をスクレイピングする

【注意点】

- Googleドライブに「ターゲットとしているスレッドURL」が1行ごとにまとまったテキストファイルをアップロードした状態で始める。

- 出来上がるものはデータフレームなので、そのままでは保存されない。適宜ファイルとして保存するなどしておく。以下ではcsv形式で保存する。

【参考ページ】

@8_hisakichi_8, 「2chのスレッドをWordCloudで可視化してみる　～スクレイピング篇～」（2022年5月5日取得，https://qiita.com/8_hisakichi_8/items/54a80f2be693dc34eb25）

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### スクレイピング本体

In [ ]:
import re
import requests, bs4
import time
import pandas as pd

res_database = []
f = open('/content/drive/MyDrive/url_list_1.txt', 'r') #URLリストを読み込む
url = f.readline()
while url:
     # txtファイルに記録されている1行目のURLのページを取得する
     res = requests.get(url)
     # デコード
     scanned_text = res.content[:1024].decode('ascii', errors='replace')
     match = re.search(r'charset=["\']?([\w-]+)', scanned_text)
     if match:
       res.encoding = match.group(1)
     else:
       res.encoding = 'utf-8'

     # HTMLを抽出する
     soup = bs4.BeautifulSoup(res.text, "html5lib")  # 2chではhtml5libを使用
     tmp_dict = {}
     # dtタグに日付などの情報。dt>fontの場合もあれば、dt>aのパターンもあるので、「レスNo.」「投稿者名」「投稿日時」はまとめて取得する
     # ddタグにレス内容が格納されている
     dddt = soup.find_all(["dd","dt"])
     for tag in dddt:
       # dtタグから「レスNo.」「日付」「投稿者名」を抽出
       if "<dt>" in str(tag):
         tmp_dict["res_num"] = re.search(r"\d{1,4} ",tag.text).group(0) # 元の文字列から最初の4桁以下の数字のみ＝「レスNo」.を抽出
         # 「日付」を抽出
         date_result = re.search(r"\d*/\d*/\d*",tag.text)
         if date_result:
           date_str = date_result.group()
           tmp_dict["date"] = date_str
         # 「投稿者名」を抽出し、不要な文字列を削除
         name_result = re.search(r"\d{1,4} .*",tag.text).group(0)
         name_result1 = re.sub(r"：(\d{2,4}).*","",name_result)
         name_result2 = re.sub(r" ◆.*","",name_result1)
         tmp_dict["name"] = re.sub(r"\d{1,4} ：","",name_result2)
       
       # ddタグからレス内容を抽出
       if "<dd>" in str(tag):
         tmp_dict["comment"] = re.sub("\n","",tag.text)

       # tmp_dictに格納した内容をres_databaseに転記
       if "date" in tmp_dict and "comment" in tmp_dict:
         res_database.append(tmp_dict)
         tmp_dict = {}
     time.sleep(2)
     url = f.readline()
     
   
f.close()
res_df = pd.DataFrame(res_database) #取得した全レスポンスのデータフレームを作成

2022年9月25日追加：データ収集後の集計作業の便宜のため、スレッドごとにidをふるようにした。

In [ ]:
import re
import requests, bs4
import time
import pandas as pd

res_database = []
f = open('/content/drive/MyDrive/url_list.txt', 'r') #URLリストを読み込む
url = f.readline()

thread_id = 0
while url:
     # txtファイルに記録されている1行目のURLのページを取得する
     res = requests.get(url)
     # デコード
     scanned_text = res.content[:1024].decode('ascii', errors='replace')
     match = re.search(r'charset=["\']?([\w-]+)', scanned_text)
     if match:
       res.encoding = match.group(1)
     else:
       res.encoding = 'utf-8'

     # HTMLを抽出する
     soup = bs4.BeautifulSoup(res.text, "html5lib")  # 2chではhtml5libを使用
     tmp_dict = {}
     # dtタグに日付などの情報。dt>fontの場合もあれば、dt>aのパターンもあるので、「レスNo.」「投稿者名」「投稿日時」はまとめて取得する
     # ddタグにレス内容が格納されている
     dddt = soup.find_all(["dd","dt"])
     for tag in dddt:
        tmp_dict["thread_id"] = thread_id
       # dtタグから「レスNo.」「日付」「投稿者名」を抽出
        if "<dt>" in str(tag):
            tmp_dict["res_num"] = re.search(r"\d{1,4}\s",tag.text).group(0) # 元の文字列から最初の4桁以下の数字のみ＝「レスNo」.を抽出
         # 「日付」を抽出
            date_result = re.search(r"\d*/\d*/\d*",tag.text)
            if date_result:
                date_str = date_result.group()
                tmp_dict["date"] = date_str
         # 「投稿者名」を抽出し、不要な文字列を削除　←　抽出後に行う
                name_result = re.search(r"\d{1,4} .*",tag.text).group()
                name_result1 = re.sub(r"：(\d{2,4}).*","",name_result)
                name_result2 = re.sub(r"\s◆.*","",name_result1)
                tmp_dict["name"] = re.sub(r"\d{1,4}\s：","",name_result2)
       
         # ddタグからレス内容を抽出
        if "<dd>" in str(tag):
            tmp_dict["comment"] = re.sub("\n","",tag.text)

         # tmp_dictに格納した内容をres_databaseに転記
        if "date" in tmp_dict and "comment" in tmp_dict:
            res_database.append(tmp_dict)
            tmp_dict = {}
     time.sleep(2)
     url = f.readline()
     thread_id += 1
     
   
f.close()
res_df = pd.DataFrame(res_database) #取得した全レスポンスのデータフレームを作成

In [ ]:
res_df

,thread_id,res_num,date,name,comment
0,0,1,2000/09/03,名無しさん＠１周年,オナニー我慢に我慢してオナニー中毒のむなしい日々から脱出しよう。夢精経験したことない人もチャ...
1,0,2,2000/09/03,名無しさん＠１周年,で、優勝者には一体どんないいことがあるの？？
2,0,3,2000/09/03,名無しさん＠１周年,>2 優勝とか関係ないよ。中毒から解き放たれ、美しい自分になれるんだ。
3,0,4,2000/09/03,名無しさん＠１周年,やりたいけど夢精しそうだから棄権します。
4,0,5,2000/09/03,名無しさん＠１周年,夢精はリタイヤじゃないんじゃないのか？？
...,...,...,...,...,...
53545,56,636,2009/03/13,クレストン（リセット）,死亡
53546,56,637,2009/03/13,わにとかげ【13日目】,生存
53547,56,638,2009/03/14,石倉流星【0日目】,たったいまビクビクしてきました また禁オナ続けます
53548,56,639,2009/03/14,わにとかげ【14日目】,生存


### データフレームを保存

In [ ]:
res_df.to_csv('/content/drive/MyDrive/PythonData/kin-ona_marathon.csv', index=False, encoding='UTF-8')